# Code for the implementation of task IRIS


### Importing libraries

In [ ]:
# For computation
import numpy as np
# For confusion matrices
from sklearn import metrics
import matplotlib.pyplot as plt
import math

## 1.

### a) Importing labeled data for training and testing

In [ ]:
class Dataset:
    def __init__(self, instances):
        data = {"training": {"targets": [], "features":[]}, "testing": {"targets": [], "features":[]}}
        labelToTarget = {"Iris-setosa": [1, 0, 0], "Iris-versicolor": [0, 1, 0], "Iris-virginica": [0, 0, 1]}
        for instance in instances:
            match instance.set:
                case 'training':
                    data["training"]["targets"].append(labelToTarget[instance.label])
                    data["training"]["features"].append(instance.features)
                case 'testing':
                    data["testing"]["targets"].append(labelToTarget[instance.label])
                    data["testing"]["features"].append(instance.features)
                    
        # convert to numpy array
        data["training"]["targets"] = np.array(data["training"]["targets"]).astype(float)
        data["training"]["features"] = np.array(data["training"]["features"]).astype(float)
        data["testing"]["targets"] = np.array(data["testing"]["targets"]).astype(float)
        data["testing"]["features"]  = np.array(data["testing"]["features"]).astype(float)
        
        self.data = data
        self.classes_names = ["Iris-setosa", "Iris-versicolor", "Iris-virginica"]
        self.feature_names = ['Sepal length [cm]', 'Sepal width [cm]', 'Petal length [cm]', 'Petal width [cm]']
        self.DESCR = "Iris dataset"

In [ ]:
class Instance:
    def __init__(self, features, label, set):
        self.features = features
        self.label = label
        self.set = set    

In [ ]:
def loadDataSet(features_list: list):
    Instances = []
    path = "Iris_TTT4275/iris.data"
    n_classes = 3
    n_training = 30
    n_testing  = 20
    with open(path) as file:
        for _ in range(n_classes):
            for _ in range(n_training):
                line = file.readline()
                line = line.split(',')
                features = []
                for i in features_list:
                    features.append(line[i]) 
                label = line[-1].strip("\n")
                training_instance = Instance(features=features, label=label, set="training")
                Instances.append(training_instance)
            for _ in range(n_testing):
                line = file.readline()
                line = line.split(',')
                features = []
                for i in features_list:
                    features.append(line[i]) 
                label = line[-1].strip("\n")
                testing_instance = Instance(features=features, label=label, set="testing")
                Instances.append(testing_instance)
    return Dataset(Instances)

In [ ]:
IRIS_Dataset = loadDataSet([0, 1, 2, 3])

### b) Training a linear classifier

#### Linear classifier equations

from the [Compendium-Part-III-Classification](Resources/Compendium_Classification.pdf)

##### Chapter 2.4 

$$
x \in \omega_j \iff g_j(x) = \max_i g_i(x)
$$
x is in the class ($\omega_j$) that has the highest value for the discriminant function ($g_j$). This is equivalent to making a partition of the whole input space, and for each point you assign the point to the class with the highest "probability" that the point belongs to that class.
The discrimination function:
$$
g_i(x) = \omega_i^T x + \omega_{io} , \qquad i = i, \dots, C 
$$
Where $\omega_i$ is the class $x$ is the feature vector, and the $\omega_{io}$ is the offset for the class i. In our case C =3, so we can write the disctimination function in compact form: 
$$
g = W x + \omega_o 
$$
where $W \in M_{CxF}(\mathbb{R})$ and $g$ and $\omega_o$ both are column vectors with size C (#classes). F stands for #features. In our case:  $C=3 \land F=4$    
When training we are aiming at finding the best values for $W$ and $\omega_o$

##### Chapter 3.2 


To get only one linear term, we do one more trick:
This trick yields the same equations only that it now gets more compact once again
$$
\begin{bmatrix} W & \omega_o \end{bmatrix} \to W\\
\begin{bmatrix} x^T & 1 \end{bmatrix} \to x \\
\implies g = Wx
$$
Loss function
$$
MSE = \frac{1}{2} \sum_{k=1}^{N}(g_k-t_k)^T(g_k-t_k)
$$
To decide how good our model is doing. Here t is the target vector (correct labeled features for the training set.)
Activation function:
$$
g_{k} = \frac{1}{1+\exp^{-z_{ik}}}, \qquad z_{ik} = Wx_k \\
$$
We should ideally use a heavyside function, but we need the derivative, and therefor we us the function up above (sigmoid function). This is used for the discrimination function.

The way to go? We want to minimize the MSE, based on choosing the W matrix. We use gradient descent.
Then we need to hyperparameters which we are tuning:
$$
\alpha: \text{Learning rate [0.0-1.0]}\\
n: \text{number of epochs (how many times we improve the W-matrix [0<])}
$$
Based on them we are using the gradient descent:
$$
\text{for i in range(1, n+1):}\\
 \qquad  \qquad W_{i+1} = W_{i} - \alpha \nabla_WMSE
$$
For the calculation of the gradient we once again have calculations from the [Compendium-Part-III-Classification](Resources/Compendium_Classification.pdf)

Using the chain rule
$$
\nabla_W MSE = \sum^{N}_{k=1}\nabla_{g_{k}} \, MSE \, \nabla_{z_{k}} \, g_k \, \nabla_W z_k\\
$$
Some simplifications
$$
\nabla_{g_k}MSE = g_k-t_k\\
\nabla_{z_k}g = g_k \circ (1-g_k)\\
\nabla_Wz_k = x_k^T\\
$$
Yields the result
$$
\nabla_W MSE = \sum^{N}_{k=1} [(g_k-t_k) \circ g_k \circ (1-g_k)]x_k^T
$$
Where $\circ$ is elementwise multiplication.

## Training phase

In [ ]:
# Activation function
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [ ]:
def TrainClassifier(dataset: Dataset, step_length: float, n_iterations, W_0: np.ndarray):
    W = W_0
    MSEs = []
    n_classes = 3
    n_training = 30
    for i in range(n_iterations):
        Gradient_MSE = np.zeros(W_0.shape)
        MSE = 0
        for index in range(n_classes * n_training):
            x_k = dataset.data["training"]["features"][index]
            x_k = np.append(x_k, 1)
            z_k = np.dot(W, x_k)
            g_k = sigmoid(z_k)
            t_k = dataset.data["training"]["targets"][index]
            MSE += 1/2 * np.dot((g_k - t_k).T,(g_k - t_k))
            Gradient_MSE += np.outer((g_k - t_k)*g_k*(np.ones((1,3))-g_k), x_k.T)
        W = W - step_length * Gradient_MSE
        MSEs.append(MSE)
    print(f"MSE in first iteration {MSEs[0]}, and last iteration {MSEs[-1]}\n")
    print(f"Our W is \n{W}\n")
    return W

In [ ]:
'''
Hyperparameters, step length and number of epochs
'''
alpha = 0.01
n = 10000
W_0 = np.zeros((3, 5))
W = TrainClassifier(IRIS_Dataset, alpha, n, W_0)

## Testing part

### Confusion Matrices.


In [ ]:
def TestClassifier(dataset: Dataset, W: np.ndarray):
    n_classes = 3
    n_training = 30
    n_testing = 20
    ConfMatrices = {"training": np.zeros((n_classes, n_classes)), "testing": np.zeros((n_classes, n_classes))}
    errors = {"training": 0, "testing": 0}
    for index in range(n_classes * n_testing):
        x_k = IRIS_Dataset.data["testing"]["features"][index]
        x_k = np.append(x_k, 1)
        t_k = IRIS_Dataset.data["testing"]["targets"][index]
        g_k = np.dot(W, x_k)
        ConfMatrices["testing"][np.argmax(t_k), np.argmax(g_k)] += 1
        if np.argmax(t_k) != np.argmax(g_k):
            errors["testing"] += 1
    
    for index in range(n_classes * n_training):
        x_k = IRIS_Dataset.data["training"]["features"][index]
        x_k = np.append(x_k, 1)
        t_k = IRIS_Dataset.data["training"]["targets"][index]
        g_k = np.dot(W, x_k)
        ConfMatrices["training"][np.argmax(t_k), np.argmax(g_k)] += 1
        if np.argmax(t_k) != np.argmax(g_k):
            errors["training"] += 1
    
    print(f"Error-rates: \nTraining: {errors['training']/(n_training*n_classes)}, Testing: {errors['testing']/(n_classes*n_testing)}\n")
    return ConfMatrices

In [ ]:
def PlotConfusionMatrix(matrix):
    fig, ax = plt.subplots(1, 2, figsize=(16,8))
    ax[0].set_title("Training")
    ax[1].set_title("Testing")
    metrics.ConfusionMatrixDisplay(confusion_matrix=matrix["training"], 
                                   display_labels=IRIS_Dataset.classes_names,
                                   ).plot(ax=ax[0], cmap="Blues")
    metrics.ConfusionMatrixDisplay(confusion_matrix=matrix["testing"],
                                   display_labels=IRIS_Dataset.classes_names,
                                   ).plot(ax=ax[1], cmap="Blues")
    plt.tight_layout()
    plt.show()


### c)

In [ ]:
ConfMatrix = TestClassifier(IRIS_Dataset, W)

PlotConfusionMatrix(ConfMatrix)

### d)

In [ ]:
path = "Iris_TTT4275/iris.data"
n_classes = 3
n_training = 20
n_testing  = 30

Instances = []

with open(path) as file:
    for _ in range(n_classes):
        for _ in range(n_testing):
            line = file.readline()
            line = line.split(',')
            features = line[0:-1]
            label = line[-1].strip("\n")
            testing_instance = Instance(features=features, label=label, set="testing")
            Instances.append(testing_instance)
            
        for _ in range(n_training):
            line = file.readline()
            line = line.split(',')
            features = line[0:-1]
            label = line[-1].strip("\n")
            training_instance = Instance(features=features, label=label, set="training")
            Instances.append(training_instance)
IRIS_Dataset = Dataset(Instances)

In [ ]:
'''
Hyperparameters, step length and number of epochs
'''
alpha = 0.01
n = 10000
W_0 = np.zeros((3, 5))

W = W_0
MSEs = []
n_classes = 3
n_training = 20
for i in range(n):
    Gradient_MSE = np.zeros(W_0.shape)
    MSE = 0
    for index in range(n_classes * n_training):
        x_k = IRIS_Dataset.data["training"]["features"][index]
        x_k = np.append(x_k, 1)
        z_k = np.dot(W, x_k)
        g_k = sigmoid(z_k)
        t_k = IRIS_Dataset.data["training"]["targets"][index]
        MSE += 1/2 * np.dot((g_k - t_k).T,(g_k - t_k))
        Gradient_MSE += np.outer((g_k - t_k)*g_k*(np.ones((1,3))-g_k), x_k.T)
    W = W - alpha * Gradient_MSE
    MSEs.append(MSE)
print(f"MSE in first iteration {MSEs[0]}, and last iteration {MSEs[-1]}\n")
print(f"Our W is {W}\n")

In [ ]:
n_classes = 3
n_training = 20
n_testing = 30
ConfMatrix = {"training": np.zeros((n_classes, n_classes)), "testing": np.zeros((n_classes, n_classes))}
for index in range(n_classes * n_testing):
    x_k = IRIS_Dataset.data["testing"]["features"][index]
    x_k = np.append(x_k, 1)
    t_k = IRIS_Dataset.data["testing"]["targets"][index]
    g_k = np.dot(W, x_k)
    ConfMatrix["testing"][np.argmax(t_k), np.argmax(g_k)] += 1

for index in range(n_classes * n_training):
    x_k = IRIS_Dataset.data["training"]["features"][index]
    x_k = np.append(x_k, 1)
    t_k = IRIS_Dataset.data["training"]["targets"][index]
    g_k = np.dot(W, x_k)
    ConfMatrix["training"][np.argmax(t_k), np.argmax(g_k)] += 1

PlotConfusionMatrix(ConfMatrix)

#TODO: Set dei ved siden av kvarandre. 

## 2

a)

In [ ]:
# for f in range(4):
#     for g in range(4):
#         if f != g:
#             plt.figure()
#             plt.scatter(IRIS_Dataset.data["training"]["features"][:,f], IRIS_Dataset.data["training"]["features"][:,g], c=IRIS_Dataset.data["training"]["targets"][:,0])
#             plt.title(f"Feature {f} vs feature {g}")
#             plt.show()

In [ ]:
n_examples = 50
paths = ["Iris_TTT4275/class_1", "Iris_TTT4275/class_2", "Iris_TTT4275/class_3"]
features = {paths[0]: [[], [], [], []], paths[1]:[[], [], [], []], paths[2]:[[], [], [], []]}
for i in range(len(paths)):
    with open(paths[i]) as file:
        for j in range(n_examples):
            line = file.readline().strip('\n').split(',')
            for k in range(len(line)):
                features[paths[i]][k].append(float(line[k]))

In [ ]:
n_bars = 6
opacity = 0.5
plt.figure(figsize=(13,13))
for index, feature in enumerate(IRIS_Dataset.feature_names):
    plt.subplot(2, 2, index+1)
    for class_nr in range(n_classes):
        plt.hist(features[paths[class_nr]][index], label=IRIS_Dataset.classes_names[class_nr], bins=n_bars, alpha=opacity)
    plt.legend()
    plt.xlabel(feature)
    plt.ylabel('count')
    
plt.show()

As Can be seen by the plots, feature 'Sepal width' is caotic and should be excluded. This is done below.

In [ ]:
IRIS_Dataset = loadDataSet([0, 2, 3])

In [ ]:
'''
Hyperparameters, step length and number of epochs
'''
alpha = 0.01
n = 10000
W_0 = np.zeros((3, 4))

W = TrainClassifier(IRIS_Dataset, alpha, n, W_0)

In [ ]:
ConfMatrix = TestClassifier(IRIS_Dataset, W)

PlotConfusionMatrix(ConfMatrix)

### With only two features:  Petal width & length

In [ ]:
IRIS_Dataset = loadDataSet([2, 3])

In [ ]:
'''
Hyperparameters, step length and number of epochs
'''
alpha = 0.01
n = 10000
W_0 = np.zeros((3, 3))

W = TrainClassifier(IRIS_Dataset, alpha, n, W_0)

In [ ]:
ConfMatrix = TestClassifier(IRIS_Dataset, W)

PlotConfusionMatrix(ConfMatrix)

### And with only one feature (Petal width):

In [ ]:
IRIS_Dataset = loadDataSet([3])

In [ ]:
'''
Hyperparameters, step length and number of epochs
'''
alpha = 0.01
n = 10000
W_0 = np.zeros((3, 2))

W = TrainClassifier(IRIS_Dataset, alpha, n, W_0)

In [ ]:
ConfMatrix = TestClassifier(IRIS_Dataset, W)

PlotConfusionMatrix(ConfMatrix)

TODO: 
* Comment on linear separability